In [ ]:
!pip install pydantic

In [2]:
import pydantic
print(pydantic.__version__)

2.8.2


# Basic Pydantic Model

In [3]:
from pydantic import BaseModel

#create a class and inherit from BaseModel
class User(BaseModel):
    id: int
    name: str
    email: str
    password: str

user = User(id=1, name="John Doe", email="john@example.com", password="password123")
user

User(id=1, name='John Doe', email='john@example.com', password='password123')

In [4]:
#Helper methods in pydantic
print(user.model_fields_set)


{'password', 'name', 'email', 'id'}


In [5]:
print(user.model_dump())

{'id': 1, 'name': 'John Doe', 'email': 'john@example.com', 'password': 'password123'}


In [6]:
print(user.model_dump_json())

{"id":1,"name":"John Doe","email":"john@example.com","password":"password123"}


In [8]:
print(user.model_json_schema())

{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'type': 'string'}, 'email': {'title': 'Email', 'type': 'string'}, 'password': {'title': 'Password', 'type': 'string'}}, 'required': ['id', 'name', 'email', 'password'], 'title': 'User', 'type': 'object'}


# Nested Models

In [9]:
from typing import List, Optional
from pydantic import BaseModel, Field

class Food(BaseModel):
    name:str
    price:float
    description:Optional[str] = None
    rating:Optional[float] = None

class Restaurant(BaseModel):
    name:str
    menu:List[Food] = []

restaurant_instance = Restaurant(
    name="ABC",
    menu=[
        {
            "name":"Pizza",
            "price":10.99,
            "description":"Delicious pizza",
            "rating":4.5
        },
        {
            "name":"Burger",
            "price":8.99,
            "description":"Juicy burger",
            "rating":4.2
        }
    ]
)

print(restaurant_instance)
print(restaurant_instance.model_dump_json())

name='ABC' menu=[Food(name='Pizza', price=10.99, description='Delicious pizza', rating=4.5), Food(name='Burger', price=8.99, description='Juicy burger', rating=4.2)]
{"name":"ABC","menu":[{"name":"Pizza","price":10.99,"description":"Delicious pizza","rating":4.5},{"name":"Burger","price":8.99,"description":"Juicy burger","rating":4.2}]}


In [ ]:
#for email validation using pydantic
!pip install pydantic[email]

In [12]:
from typing import List
from pydantic import BaseModel, Field, EmailStr, conlist, PositiveInt, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip: str
    country: str

class Employee(BaseModel):
    name: str
    age: int
    email: EmailStr
    address: Address
    position: str
    salary: float
    hire_date: str

class Owners(BaseModel):
    name: str
    age: int
    email: EmailStr
    address: Address
    company: str
    position: str
    salary: float
    hire_date: str

class Restaurant(BaseModel):
    name: str = Field(...,pattern="^[a-zA-Z0-9-' ]+$")
    address: Address
    menu: List[str]
    employees: conlist(Employee,min_length=2)
    owners: List[Owners]
    number_of_seats: PositiveInt
    website:HttpUrl

In [16]:
restaurant_instance = Restaurant(
    name="Taco Bell",
    address={
        "street": "123 Main St",
        "city": "Anytown",
        "state": "CA",
        "zip": "12345",
        "country":"USA"
    },
    menu=["Item1","Item2","Item3"],
    number_of_seats=100,
    website="https://tacobell.com"
)


ValidationError: 2 validation errors for Restaurant
employees
  Field required [type=missing, input_value={'name': 'Taco Bell', 'ad... 'https://tacobell.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
owners
  Field required [type=missing, input_value={'name': 'Taco Bell', 'ad... 'https://tacobell.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

# Field Validators

In [19]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    password: str

    @field_validator('password')
    @classmethod
    def  validate_password(cls, value:str) -> str:
        if len(value) < 8:
            raise ValueError('Password must be at least 8 characters long')
        return value
    
try:
    owner_instance = Owner(name="John Doe", email="john@example.com", password="weak")
except ValueError as e:
    print(e)

1 validation error for Owner
password
  Value error, Password must be at least 8 characters long [type=value_error, input_value='weak', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error


# Model Validator
- use to validate the model, instead of validating individual fields